# Prediction and Information: The Math Behind Loss Functions

기계 학습 모델(예: 분류기)을 학습할 때는 모델의 예측이 실제 결과와 얼마나 일치하는지를 측정하는 기준이 필요하다. 이때 사용하는 것이 바로 **손실 함수(loss function)** 이다. 특히 분류 문제에서 널리 쓰이는 손실 함수들은 *정보 이론(information theory)* 에 깊은 기반을 두고 있다. 이 글에서는 **엔트로피(entropy)**, **교차 엔트로피(cross-entropy)**, **KL 발산(Kullback–Leibler divergence)** 이 어떻게 손실 함수의 수학적 토대를 이루는지를 설명한다.

---

## 1. 정보량과 엔트로피: 불확실성의 정량화

**정보량(information content)** 은 어떤 사건이 얼마나 예기치 못한지를 수치로 나타낸 개념이다. 확률이 $p$인 사건의 정보량은 다음과 같이 정의된다:

$$
I = -\log p
$$

로그의 밑이 2이면 단위는 비트(bits)다. 확률이 높을수록 정보량은 낮고(예상 가능성이 큼), 확률이 낮을수록 정보량은 크다(예상 밖의 사건). 예를 들어, 동전을 던져 앞면이 나올 확률이 0.5이면 정보량은 $1$비트이며, 주사위를 던져 특정 숫자가 나올 확률 $\frac{1}{6}$은 약 $2.58$비트의 정보량을 가진다.

**엔트로피(entropy)** 는 확률 분포 전반의 평균 정보량으로, 변수의 불확실성을 측정한다. 이산 확률 변수 $X$가 가능한 값 $x_1, x_2, \dots, x_n$을 갖고, 각 확률이 $p_j = P(X = x_j)$일 때, 엔트로피는 다음과 같이 정의된다:

$$
H(X) = -\sum_{j=1}^{n} p_j \log p_j
$$

즉, 가능한 모든 사건에 대해 정보량의 기대값을 계산한 것이다. 엔트로피는 분포가 균일할 때 가장 크며, 한 사건만 확률 1이고 나머지가 0일 때는 0이 된다.

> 💡 **참고:** 정보 이론에서는 $\log_2$를 사용해 비트 단위로 표현하지만, 머신러닝에서는 자연로그(밑 $e$)를 써서 단위를 ‘nat’로 나타내는 경우가 많다. 로그의 밑은 수치만 바꿀 뿐 최적화 결과에는 영향을 미치지 않는다.

---

## 2. 교차 엔트로피: 예측 분포와 실제 분포의 차이

모델이 출력하는 확률 분포 $Q$와 실제 정답 분포 $P$ 간의 차이를 측정하는 대표적인 지표가 **교차 엔트로피(cross-entropy)** 다. 이는 다음과 같이 정의된다:

$$
H(P, Q) = -\sum_{x} p(x) \log q(x)
$$

이 수식은 실제 분포 $P$에서 샘플을 뽑고, 예측 분포 $Q$에 따라 부호화할 때 필요한 평균 정보량을 나타낸다.

* $Q = P$일 경우: $H(P, Q) = H(P)$ → 모델이 정확히 맞췄을 때 손실은 최소
* $Q \neq P$일 경우: $H(P, Q) > H(P)$ → 예측 분포가 실제와 다를수록 손실 증가

### 📌 분류 문제에서의 적용

분류 문제에서는 실제 분포 $P$가 one-hot 형식으로 주어진다. 예를 들어, 정답 클래스가 A라면 $P(A)=1$, 나머지는 0이다. 이 경우 교차 엔트로피는 다음과 같이 단순화된다:

$$
H(P, Q) = -\log q_y
$$

즉, **손실 값은 정답 클래스에 대한 예측 확률의 음의 로그**가 된다.

* 예측 확률이 높으면 손실은 작아진다.
* 예측 확률이 낮으면 손실은 커진다.

### 🧮 실제 예시로 보는 단순화

클래스가 3개(A, B, C)이고, 실제 정답이 B라고 하자. 이때 실제 분포는 one-hot으로 다음과 같다:

$$
P = [0, 1, 0]
$$

모델이 예측한 확률 분포가 다음과 같다면:

$$
Q = [0.2, 0.7, 0.1]
$$

교차 엔트로피는 원래 다음과 같은 시그마 형태지만:

$$
H(P, Q) = - \sum_{i} p_i \log q_i
$$

one-hot 분포에서는 정답 클래스(B)에 해당하는 항만 1이고 나머지는 0이므로, 실제 계산은 다음과 같이 단순화된다:

$$
H(P, Q) = - (0 \cdot \log 0.2 + 1 \cdot \log 0.7 + 0 \cdot \log 0.1) = -\log 0.7
$$

즉, 다른 클래스의 예측값(0.2, 0.1)은 0과 곱해져서 자연스럽게 사라진다. 이로 인해 시그마 없이 정답 클래스 하나의 확률만 남게 되는 것이다.

### ✅ 교차 엔트로피가 널리 쓰이는 이유

* 항상 0보다 크거나 같다
예측이 완벽하면 손실 값은 0에 가까워지고, 틀릴수록 손실이 커진다.
즉, 숫자로 봤을 때 손실이 **얼마나 잘 맞췄는지**를 잘 표현해준다.

* 틀릴수록 손실이 급격히 커진다 로그 함수의 성질 때문에 정답 확률을 아주 낮게 예측하면 손실이 폭발적으로 커진다.  
→ 이 덕분에 모델은 틀린 예측에 민감하게 반응해서 빠르게 학습할 수 있다.

* 통계적으로도 의미가 있다 (최대우도 추정과 같음)
모델이 실제 데이터를 가장 잘 설명하는 방법은
정답일 확률을 최대한 높이게 만드는 것인데,
이게 바로 교차 엔트로피를 줄이는 것과 똑같다.  
→ 수학적으로도 ‘최고의 예측 모델’을 찾는 방향과 일치한다.

> ℹ️ **Binary vs. Categorical:**
>
> * 이진 분류에서는 $-\left[y \log p + (1 - y) \log(1 - p)\right]$ 형태로 나타나는 로지스틱 손실이 사용된다.
> * 다중 클래스 분류에서는 softmax + 교차 엔트로피가 함께 사용된다.

---

## 3. KL divergence: 예측 분포의 정보적 비효율성

**KL 발산(Kullback–Leibler divergence)** 은 예측 분포 $Q$가 실제 분포 $P$와 얼마나 다른지를 측정하는 정보 이론적 지표다. 정의는 다음과 같다:

$$
D_{KL}(P \parallel Q) = \sum_x p(x) \log \frac{p(x)}{q(x)}
$$

또는 다음처럼 쓸 수 있다:

$$
D_{KL}(P \parallel Q) = H(P, Q) - H(P)
$$

즉, **KL 발산은 교차 엔트로피에서 엔트로피를 뺀 값**이며, 예측 분포를 사용함으로써 발생하는 추가적인 정보 낭비를 의미한다.

* $P = Q$일 때 KL 발산은 0 (정보 낭비 없음)
* $P \neq Q$일수록 발산 값 증가 (비효율적 인코딩)

> ⚠️ KL 발산은 **비대칭**이다: $D_{KL}(P\parallel Q) \neq D_{KL}(Q\parallel P)$

### 📌 분류 문제에서의 의미

$P$가 one-hot 분포인 경우 $H(P) = 0$이므로,

$$
D_{KL}(P \parallel Q) = H(P, Q)
$$

즉, **교차 엔트로피 손실을 최소화하는 것**은 **KL 발산을 최소화하는 것과 동일**하다. 이는 예측 분포 $Q$를 실제 분포 $P$에 최대한 가깝게 만들도록 유도하는 것이다.

---

## 4. 실전 학습에서의 적용: 배치 단위 손실과 역전파

실제 딥러닝 모델 학습에서는 손실 함수를 한 번에 하나의 샘플에 대해서만 계산하지 않고, **여러 개의 샘플을 하나의 배치(batch)** 로 묶어 동시에 처리한다. 예를 들어 배치 크기가 32라면, 모델은 32개의 입력에 대해 각각 예측을 수행하고, 각 샘플에 대해 교차 엔트로피 손실 값을 계산한다.

이때 중요한 점은 다음과 같다:

* **각 샘플의 손실 값들은 평균 또는 합산**되어 하나의 스칼라 손실로 집계된다.
* 이 최종 손실 값 하나에 대해 **역전파(backpropagation)** 가 수행되어, 모델의 모든 파라미터에 대한 그래디언트가 계산된다.

즉, 실제 학습 과정은 다음과 같이 진행된다:

1. $x_1, x_2, \dots, x_n$ → 각각의 손실 $L_1, L_2, \dots, L_n$ 계산
2. 손실 평균 $L = \frac{1}{n} \sum_{i=1}^{n} L_i$
3. $L$에 대해 역전파: $L.\texttt{backward()}$

이 구조는 모델이 더 안정적으로 학습되도록 도와주며, 다양한 하드웨어 가속(예: GPU)의 병렬 계산 이점을 최대한 활용하게 한다.

---

## 5. 결론: 손실 함수와 정보 이론의 연결

정보 이론은 손실 함수의 수학적 기반을 제공하며, 예측 정확도를 정량적으로 평가할 수 있는 강력한 도구를 제시한다.

| 개념                               | 의미                        | 수학적 정의                    |
| -------------------------------- | ------------------------- | ------------------------- |
| **엔트로피** $H(P)$                  | 실제 분포의 평균 불확실성            | $-\sum p \log p$          |
| **교차 엔트로피** $H(P, Q)$            | 예측 분포로 실제를 인코딩할 때의 평균 정보량 | $-\sum p \log q$          |
| **KL 발산** $D_{KL}(P\parallel Q)$ | 예측 분포의 정보적 낭비             | $\sum p \log \frac{p}{q}$ |

**교차 엔트로피 손실 함수**는 단순한 경험적 수치가 아니라, 모델이 얼마나 ‘정보 효율적으로’ 실제를 설명하는지를 보여준다.  
따라서 이 손실을 줄인다는 것은 예측이 더 정밀해지고, 정보적 낭비가 적은 방향으로 모델이 발전하고 있다는 의미다.

---
